In [1]:
import torch
import torch.nn as nn

In [ ]:
class Block(nn.Module):
    def __init__(self, in:channel, out_channel, identity_downsample=None, stride=1):
        super(Block, self).__init__()
        self.expansion=4
        self.conv1=nn.Conv2d